In [ ]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from decimal import Decimal

In [ ]:
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler

In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
%load_ext autoreload
%autoreload 2
from strategy.Positions import UniV3Position, BiCurrencyPosition
from strategy.Portfolio import Portfolio
from strategy.Data import PoolDataUniV3, RawDataUniV3, SyntheticData
from strategy.Backtest import Backtest
from strategy.History import PortfolioHistory
from strategy.Strategies import BiCurrencyPassive, BiCurrencyActive, UniV3Passive, UniV3Active
from strategy.Viewers import PotrfolioViewer, RebalanceViewer, UniswapViewer, LiqudityViewer
from strategy.MultiStrategy import MultiStrategy
from strategy.primitives import Pool, Token, Fee, Frequency

In [ ]:
def tick_to_price(tick, decimal_diff=10):
    price = np.power(1.0001, tick) / 10 ** decimal_diff
    return price

def price_to_tick(price, decimal_diff=10):
    tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
    return int(round(tick))

In [ ]:
pool = Pool(Token.USDC, Token.WETH, Fee.LOW)

In [ ]:
data = RawDataUniV3(pool).load_from_folder()

In [ ]:
LiqudityViewer(data).draw_plot()

In [ ]:
data.swaps = data.swaps[0:100000]

In [ ]:
def preprocess_transactions(df_transactions, df_swaps):
    df_transactions = df_transactions.rename(columns={'hash': 'tx_hash'})    
    df_transactions_slice = df_transactions.loc[df_transactions['tx_hash'].isin(df_swaps['tx_hash'])]
    
    df_transactions_slice['tx_fee'] = df_transactions_slice.apply(lambda x: (Decimal(x['gas']) * Decimal(x['gas_price'])) / Decimal(10**18), axis=1)
    df_transactions_slice['tx_fee'] = df_transactions_slice['tx_fee'].astype(float)
    
    df_transactions_slice['gas_price'] = df_transactions_slice.apply(lambda x: Decimal(x['gas_price']) / Decimal(10**9), axis=1)
    
    
    swaps_full = df_swaps.reset_index().merge(df_transactions_slice, on=['tx_hash', 'block_number'], how='left')    
    swaps_full = swaps_full.set_index('timestamp')
    return swaps_full

In [ ]:
df_transactions = pd.read_csv('../data/all_transactions.csv')

In [ ]:
swaps_full = preprocess_transactions(df_transactions, data.swaps)

In [ ]:
base = swaps_full.resample('5min').size().to_frame('tx_num')
base['mean_price'] = swaps_full.resample('5min')['price'].mean().ffill()
base['total_amount1'] = swaps_full['amount1'].abs().resample('5min').sum()
base['tx_fee'] = swaps_full.resample('5min')['tx_fee'].mean().fillna(0)

In [ ]:
base

## Bi-currency passive strategy

In [ ]:
bi_curr_strat = BiCurrencyPassive(pool)

In [ ]:
b = Backtest(bi_curr_strat)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
stats_df_sbc = portfolio_history.portfolio_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)

In [ ]:
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
f1, f2, f3, f4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
f1

In [ ]:
f2

In [ ]:
f3

In [ ]:
f4

## Bi-currency active strat 

In [ ]:
lower_0 = data.swaps['price'].min()
upper_0 = data.swaps['price'].max()

bi_curr_strat = BiCurrencyActive(1200, 400, lower_0, upper_0, pool, 0.01,  0.0002, 0.0002)

In [ ]:
b = Backtest(bi_curr_strat)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
stats_df_bicur_active = portfolio_history.portfolio_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

## Passive UniV3 strategy

In [ ]:
univ3_passive = UniV3Passive(12, 18, pool, 0.01)

In [ ]:
b = Backtest(univ3_passive)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
uv = UniswapViewer(uni_history)
uni_interval_fig = uv.draw_intervals(data.swaps)

In [ ]:
uni_interval_fig

In [ ]:
uni_history.get_coverage( data.swaps)

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig3

In [ ]:
fig2

In [ ]:
fig4

### Composit strategy UniV3Passive + BiCurrensy Active

In [ ]:
bi_curr_strat = BiCurrencyActive(600, 200, 12, 18, pool, 0.01,  0.0002, 0.0002)
univ3_passive = UniV3Passive(12, 18, pool, 0.01)

In [ ]:
ms = MultiStrategy('Multi', [bi_curr_strat, univ3_passive])

In [ ]:
portfolio_history, rebalance_history, uni_history = Backtest(ms).backtest(data.swaps)

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
uv = UniswapViewer(uni_history)
uni_interval_fig = uv.draw_intervals(data.swaps)

In [ ]:
uni_interval_fig

In [ ]:
uni_history.get_coverage( data.swaps)

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig2

In [ ]:
fig4

## Active UniV3 strategy

In [ ]:
def rolling_normalized(df, col, window_size):
    df_rolling = df[[col]].rolling(window_size).mean()
    scaler = StandardScaler()
    df_rolling_normed = scaler.fit_transform(df_rolling)
    df_rolling[col] = df_rolling_normed
    return df_rolling[col]

In [ ]:
signal_1 = np.sign(rolling_normalized(base, 'tx_fee', 288 * 14) - rolling_normalized(base, 'tx_fee', 288 * 3))

In [ ]:
lower_0 = 11
upper_0 = 19

mint_tolerance = 10
grid_width = 60
width_num = 2
# burn_num = 12

# burn_tolerance = grid_width * grid_num - mint_tolerance
# burn_tolerance = grid_width * (grid_num + burn_num) - mint_tolerance
burn_tolerance = 360

univ3_active = UniV3Active(signal_1,  burn_tolerance, grid_width, width_num, 200, lower_0, upper_0, pool, 0.01, 0.0002, 0.0002)

In [ ]:
bi_curr_strat = BiCurrencyActive(600, 200, 12, 18, pool, 0.01,  0.0002, 0.0002)
ms = MultiStrategy('Multi', [bi_curr_strat, univ3_active])

In [ ]:
portfolio_history, rebalance_history, uni_history = Backtest(ms).backtest(data.swaps)

In [ ]:
# stats_df_univ3_active = portfolio_history.portfolio_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
fig1, fig2, fig3, fig4 = PotrfolioViewer(portfolio_history).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

In [ ]:
uni_history.get_coverage(data.swaps)

In [ ]:
uv = UniswapViewer(uni_history)

In [ ]:
uv.draw_intervals(data.swaps)

## Aggregate results

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Portfolio Values
fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=stats_df_sbc.index,
#         y=stats_df_sbc['portfolio_value_to_y'],
#         name="Bi-currency Simple",
#     ))

fig.add_trace(
    go.Scatter(
        x=stats_df_bicur_active.index,
        y=stats_df_bicur_active['portfolio_value_to_y'],
        name='Bi-currency balanced 2100',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_passive.index,
        y=stats_df_univ3_passive['portfolio_value_to_y'],
        name='UniV3 Passive',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_active.index,
        y=stats_df_univ3_active['portfolio_value_to_y'],
        name='UniV3 Active',
    ))

fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="Value to Y")
fig.update_layout(title='Portfolio Values to Y')

In [ ]:
# Portfolio Performances
fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=stats_df_sbc.index,
#         y=stats_df_sbc['portfolio_performance_to_y_to_year'],
#         name="Bi-currency Simple",
#     ))

fig.add_trace(
    go.Scatter(
        x=stats_df_bicur_active.index,
        y=stats_df_bicur_active['portfolio_performance_to_y_to_year'],
        name='Bi-currency balanced 2100',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_passive.index,
        y=stats_df_univ3_passive['portfolio_performance_to_y_to_year'],
        name='UniV3 Passive',
    ))

fig.add_trace(
    go.Scatter(
        x=stats_df_univ3_active.index,
        y=stats_df_univ3_active['portfolio_performance_to_y_to_year'],
        name='UniV3 Active',
    ))

fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="Performance in Y")
fig.update_layout(title='Portfolio Performances in Y')

In [ ]:
signal_1 = np.sign(rolling_normalized(base, 'tx_num', 288 * 14) - rolling_normalized(base, 'tx_num', 288 * 3))

In [ ]:
signal_1 = signal_1.fillna(0)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(
        x=signal_1.index,
        y=signal_1,
        name="signal",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'tx_num', 288 * 3),
        name="tx_num sum, narrow",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'tx_num', 288 * 14),
        name="tx_num sum, wide",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=base['mean_price'],
        name="Price",
    ), secondary_y=True)



In [ ]:
signal_2 = np.sign(rolling_normalized(base, 'tx_fee', 288 * 9) - rolling_normalized(base, 'tx_fee', 288 * 3))

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=signal_2.index,
        y=signal_2,
        name="signal",
    ))


fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'tx_fee', 288 * 3),
        name="tx_fee sum, narrow",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'tx_fee', 288 * 9),
        name="tx_fee sum, wide",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=base['mean_price'],
        name="Price",
    ), secondary_y=True)



In [ ]:
signal_3 = np.sign(rolling_normalized(base, 'total_amount1', 288 * 14) - rolling_normalized(base, 'total_amount1', 288 * 3))

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=signal_3.index,
        y=signal_3,
        name="signal",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'total_amount1', 288 * 3),
        name="total_amount sum, narrow",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'total_amount1', 288 * 14),
        name="total_amount sum, wide",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=base['mean_price'],
        name="Price",
    ), secondary_y=True)


In [ ]:
signal_4 = np.sign(rolling_normalized(base, 'mean_price', 288 * 7) - rolling_normalized(base, 'mean_price', 288 * 3))

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=signal_4.index,
        y=signal_4,
        name="signal",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'mean_price', 288 * 3),
        name="total_amount sum, narrow",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=rolling_normalized(base, 'mean_price', 288 * 7),
        name="total_amount sum, wide",
    ))

fig.add_trace(
    go.Scatter(
        x=base.index,
        y=base['mean_price'],
        name="Price",
    ), secondary_y=True)

In [ ]:
signal_5 = signal_1 + signal_2 
signal_5 = signal_5 == signal_5.max()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=signal_4.index,
        y=signal_4,
        name="signal",
    ))


fig.add_trace(
    go.Scatter(
        x=base.index,
        y=base['mean_price'],
        name="Price",
    ), secondary_y=True)


In [ ]:
signal_4

In [ ]:
for idx, row in swaps[1000:1010].iterrows():
    print(idx)
    print(signal_4[])

In [ ]:
signal_4.index[signal_4.index.get_loc(idx, method='nearest')]

In [ ]:
signal_4[pd.Timestamp('2021-11-04 04:15:00')]

In [ ]:
signal_4